<a href="https://colab.research.google.com/github/LeeJiwoo982/DL_study/blob/main/231012_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
train_X = [[1,2,3,4,5],
           [2,4,6,8,10],
           [1,3,5,7,9],
           [0,2,4,6,8]]
print(np.shape(train_X))

(4, 5)


In [3]:
train_X = np.array(train_X, dtype=np.float32)
print(train_X.shape)

(4, 5)


RNN은 3차원 tensor로 값을 입력받아 3차원으로 변환

In [4]:
train_X = np.array([train_X],dtype=np.float32)
print(train_X.shape)

(1, 4, 5)


중요한 파라미터인 return_sequences와 return_state 두 파라미터 default값은 False

* return_sequence=False 일 떄 마지막 시점 hidden state만 출력

1) 그렇다면 return_sequence가 True라면?

In [5]:
hidden_size = 3
cell = layers.SimpleRNNCell(units=hidden_size)
rnn = layers.RNN(cell, return_sequences=True, return_state=False)
hidden_state=rnn(train_X)

print('train_X: {} \t shape : {}'.format(train_X, train_X.shape))
print('hidden_state : {} \t shape : {}'.format(hidden_state, hidden_state.shape))

#모든 시점의 hidden_state가 출력된다.

train_X: [[[ 1.  2.  3.  4.  5.]
  [ 2.  4.  6.  8. 10.]
  [ 1.  3.  5.  7.  9.]
  [ 0.  2.  4.  6.  8.]]] 	 shape : (1, 4, 5)
hidden_state : [[[ 0.18354577 -0.99169743 -0.99998945]
  [-0.5377328  -0.9997329  -1.        ]
  [-0.36032334 -0.99967974 -1.        ]
  [ 0.08463638 -0.9989472  -1.        ]]] 	 shape : (1, 4, 3)


2)return sequence=False

In [6]:
cell = layers.SimpleRNNCell(units=hidden_size)
rnn = layers.RNN(cell, return_sequences=False, return_state=False)
hidden_state=rnn(train_X)

print('train_X: {} \t shape : {}'.format(train_X, train_X.shape))
print('hidden_state : {} \t shape : {}'.format(hidden_state, hidden_state.shape))
#마지막 시점의 hidden_state 출력

train_X: [[[ 1.  2.  3.  4.  5.]
  [ 2.  4.  6.  8. 10.]
  [ 1.  3.  5.  7.  9.]
  [ 0.  2.  4.  6.  8.]]] 	 shape : (1, 4, 5)
hidden_state : [[-0.9914199  0.9998291 -0.4724366]] 	 shape : (1, 3)


return_state = True

In [9]:
ell = layers.SimpleRNNCell(units=hidden_size)
rnn = layers.RNN(cell, return_sequences=True, return_state=True)
hidden_state, last_state=rnn(train_X)


print('hidden_state : {} \t shape : {}'.format(hidden_state, hidden_state.shape))
print('last_state: {} \t shape : {}'.format(last_state, last_state.shape))

hidden_state : [[[-0.9948807   0.99767476  0.7119209 ]
  [-0.9999979   0.9999994   0.6169265 ]
  [-0.9998914   0.9999921   0.16341244]
  [-0.9914199   0.9998291  -0.4724366 ]]] 	 shape : (1, 4, 3)
last_state: [[-0.9914199  0.9998291 -0.4724366]] 	 shape : (1, 3)


In [10]:
cell = layers.SimpleRNNCell(units=hidden_size)
rnn = layers.RNN(cell, return_sequences=False, return_state=True)
hidden_state, last_state=rnn(train_X)


print('hidden_state : {} \t shape : {}'.format(hidden_state, hidden_state.shape))
print('last_state: {} \t shape : {}'.format(last_state, last_state.shape))

hidden_state : [[ 0.75613755  0.99999857 -1.        ]] 	 shape : (1, 3)
last_state: [[ 0.75613755  0.99999857 -1.        ]] 	 shape : (1, 3)


"토마토"를 학습하자  
input='토토마를자먹'  
output='토마토를먹자'  

In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNNCell, Dense, TimeDistributed, RNN

idx2char = ['토', '마', '를', '먹', '자']

x_data = [[0,0,1,2,4,3]] #토 토 마 를 자 먹
y_data = [[0,1,0,2,3,4]] #토 마 토 를 먹 자/정답을 알려줌

num_classes = 5 #토큰의 갯수
input_dim = 5
sequence_len = 6 #len(x_data)/결과의 길이
learning_rate = 0.1 #학습률

데이터 변환 - 원핫인코딩

In [13]:
x_one_hot = tf.keras.utils.to_categorical(x_data, num_classes=num_classes)
y_one_hot = tf.keras.utils.to_categorical(y_data, num_classes=num_classes)

x_one_hot

array([[[1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0.]]], dtype=float32)

In [14]:
y_one_hot

array([[[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]]], dtype=float32)

In [15]:
x_one_hot.shape
#시퀀스수, 시퀀스길이, dim 사이즈

(1, 6, 5)

모델링

In [17]:
model = Sequential()
cell = SimpleRNNCell(units=num_classes, input_shape=(sequence_len, input_dim))

model.add(RNN(cell=cell,
            return_sequences=True,
            return_state=False,
            input_shape=(sequence_len, input_dim)))
model.add(TimeDistributed(Dense(units=num_classes, activation='softmax')))

model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              metrics=['acc'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rnn_7 (RNN)                 (None, 6, 5)              55        
                                                                 
 time_distributed_1 (TimeDi  (None, 6, 5)              30        
 stributed)                                                      
                                                                 
Total params: 85 (340.00 Byte)
Trainable params: 85 (340.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
model.fit(x_one_hot, y_one_hot, epochs=10)

Epoch 1/10
1/1 [==============================] - 0s 14ms/step - loss: 0.3470 - acc: 1.0000
Epoch 2/10
1/1 [==============================] - 0s 13ms/step - loss: 0.2458 - acc: 1.0000
Epoch 3/10
1/1 [==============================] - 0s 12ms/step - loss: 0.1702 - acc: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 13ms/step - loss: 0.1188 - acc: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 12ms/step - loss: 0.0849 - acc: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 11ms/step - loss: 0.0621 - acc: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 10ms/step - loss: 0.0462 - acc: 1.0000
Epoch 8/10
1/1 [==============================] - 0s 10ms/step - loss: 0.0348 - acc: 1.0000
Epoch 9/10
1/1 [==============================] - 0s 10ms/step - loss: 0.0266 - acc: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 12ms/step - loss: 0.0206 - acc: 1.0000


In [23]:
pred = model.predict(x_one_hot)
pred

1/1 [==============================] - 0s 24ms/step


array([[[9.9472767e-01, 1.7439260e-03, 1.1543852e-03, 2.1856215e-03,
         1.8835289e-04],
        [2.2314910e-03, 9.9101794e-01, 1.5582709e-03, 5.0080935e-03,
         1.8420462e-04],
        [9.9822515e-01, 2.9148856e-05, 6.7806576e-04, 3.8572657e-04,
         6.8202772e-04],
        [3.8647975e-03, 2.8336295e-03, 9.8636729e-01, 6.2194569e-03,
         7.1484916e-04],
        [4.1395137e-03, 7.0697027e-03, 4.0635807e-03, 9.6790946e-01,
         1.6817739e-02],
        [3.4962846e-03, 1.1354260e-04, 1.0670878e-04, 3.0965544e-02,
         9.6531790e-01]]], dtype=float32)

In [24]:
for i, word in enumerate(pred):
    print(" ".join([idx2char[c] for c in np.argmax(word, axis = 1)]))

토 마 토 를 먹 자
